In [2]:
import pandas as pd

train = pd.read_csv('data/train.csv')
train.drop(['ID'], axis=1, inplace=True)
train.columns = ['sentence', 'type', 'polarity', 'tense', 'certainty', 'label']
train.drop_duplicates(subset=['sentence'], inplace=True)
train

,sentence,type,polarity,tense,certainty,label
0,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...
16535,"이에 따라 대형 콘서트부터 야외 페스티벌, 실내 공연 등 연이어 오픈 소식이 들려오...",사실형,긍정,현재,확실,사실형-긍정-현재-확실
16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실


In [68]:
train.label.value_counts()

사실형-긍정-과거-확실     7098
사실형-긍정-현재-확실     4733
추론형-긍정-현재-확실     1099
사실형-긍정-미래-확실      648
추론형-긍정-과거-확실      334
                 ... 
대화형-미정-과거-확실        1
예측형-미정-현재-확실        1
예측형-부정-현재-불확실       1
예측형-부정-과거-확실        1
예측형-미정-과거-확실        1
Name: label, Length: 64, dtype: int64

In [69]:
import random
true_pos_past_cert = list(train.loc[(train.label == '사실형-긍정-과거-확실')].index)
true_pos_past_cert = random.sample(true_pos_past_cert, len(true_pos_past_cert) // 3)
true_pos_prsnt_cert = list(train.loc[(train.label == '사실형-긍정-현재-확실')].index)
true_pos_prsnt_cert = random.sample(true_pos_prsnt_cert, len(true_pos_prsnt_cert) // 2)



In [70]:
make_cnt_by_label = ((train.label.value_counts().std() - train.label.value_counts()) / train.label.value_counts()).round()

In [71]:
type_rare = train.loc[train.type != '사실형']
polarity_rare = train.loc[train.polarity != '긍정']
tense_rare = train.loc[train.tense == '미래']
certainty_rare = train.loc[train.certainty == '불확실']
print(len(type_rare), len(polarity_rare), len(tense_rare), len(certainty_rare))

2976 748 1642 1345


In [72]:
from eda import EDA
from collections import deque
import random
from tqdm.auto import tqdm

synth = deque()

for i, sentence, *labels, label in train.itertuples():
    sentence2make = int(make_cnt_by_label[label])
    if sentence2make > 0:
        new_sentences = EDA(sentence=sentence,
                            alpha_sr=0.1,
                            alpha_ri=0.2,
                            alpha_rs=0.2,
                            p_rd=0.1,
                            num_aug=sentence2make)

        for new_sentence in new_sentences:
            synth.append([new_sentence, *labels, label])

    else:
        if label == '사실형-긍정-과거-확실' and i in true_pos_past_cert:
            synth.append([sentence, *labels, label])
        elif label == '사실형-긍정-현재-확실' and i in true_pos_prsnt_cert:
            synth.append([sentence, *labels, label])


In [75]:
synth = pd.DataFrame(synth, columns = ['sentence', 'type', 'polarity', 'tense', 'certainty', 'label'])
len(synth)


69225

In [76]:
synth.label.value_counts()

사실형-긍정-현재-확실     2366
사실형-긍정-과거-확실     2366
사실형-긍정-미래-확실     1296
예측형-긍정-미래-불확실    1128
사실형-부정-과거-확실     1098
                 ... 
추론형-긍정-미래-불확실    1002
추론형-긍정-과거-확실     1002
사실형-긍정-과거-불확실    1001
사실형-긍정-현재-불확실     995
사실형-긍정-미래-불확실     944
Name: label, Length: 63, dtype: int64

In [13]:
train.label.value_counts()

사실형-긍정-과거-확실     7098
사실형-긍정-현재-확실     4733
추론형-긍정-현재-확실     1099
사실형-긍정-미래-확실      648
추론형-긍정-과거-확실      334
                 ... 
대화형-미정-과거-확실        1
예측형-미정-현재-확실        1
예측형-부정-현재-불확실       1
예측형-부정-과거-확실        1
예측형-미정-과거-확실        1
Name: label, Length: 64, dtype: int64

In [16]:
import random

random.choice(train.loc[train.label == '사실형-긍정-과거-확실'])

KeyError: 6751